In [1]:
# Load necessary libraries
import statsmodels.api as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
# Set up FRED data connection
# Instructions for getting your FRED API key available at these websites
# https://pypi.org/project/fredapi/ 
# http://api.stlouisfed.org/api_key.html
from fredapi import Fred
fred = Fred(api_key='8f59dec25e3eb3275581b6d50ddbc63b')

# OECD Total share price index 
temp = fred.get_series('SPASTT01USM661N', observation_start='1960-01-01', observation_end='2023-07-01')
df_total_share_price_idx = pd.DataFrame(data = temp, columns=['Total_Price_Index'])
df_total_share_price_idx = df_total_share_price_idx.rename_axis('Date').reset_index()
print(df_total_share_price_idx.head(5))

# Annual dividends
temp = fred.get_series('A2218C1A027NBEA', observation_start='1929-01-01', observation_end='2021-01-01')
df_dividends = pd.DataFrame(data = temp, columns=['Total_Dividends_Paid'])
df_dividends = df_dividends.rename_axis('Date').reset_index()
print(df_dividends.head(5))

# Dividends are measured in total annual dollars. Convert to index, base 2015 to align with share price data
div_base = np.float64(df_dividends.loc[df_dividends["Date"] == "2015-01-01", "Total_Dividends_Paid"])
df_dividends['Total_Dividends_Index'] =  ((df_dividends['Total_Dividends_Paid']/12)/div_base)*100
print(df_dividends.head(5))

# Load CPI 
temp = fred.get_series('CPIAUCSL', observation_start='1960-01-01', observation_end='2023-07-01')
df_cpi = pd.DataFrame(data = temp, columns=['CPI'])
df_cpi = df_cpi.rename_axis('Date').reset_index()

cpi_base = np.float64(df_cpi.loc[df_cpi["Date"] == "2023-07-01", "CPI"])
df_cpi['CPI_ADJ'] = cpi_base / df_cpi['CPI']
print(df_cpi.head(5))

        Date  Total_Price_Index
0 1960-01-01           3.418156
1 1960-02-01           3.255919
2 1960-03-01           3.205570
3 1960-04-01           3.233542
4 1960-05-01           3.205570
        Date  Total_Dividends_Paid
0 1946-01-01                 8.021
1 1947-01-01                 9.309
2 1948-01-01                10.647
3 1949-01-01                10.942
4 1950-01-01                13.076
        Date  Total_Dividends_Paid  Total_Dividends_Index
0 1946-01-01                 8.021               0.037506
1 1947-01-01                 9.309               0.043528
2 1948-01-01                10.647               0.049785
3 1949-01-01                10.942               0.051164
4 1950-01-01                13.076               0.061143


/tmp/ipykernel_248032/1852250754.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  div_base = np.float64(df_dividends.loc[df_dividends["Date"] == "2015-01-01", "Total_Dividends_Paid"])


        Date    CPI    CPI_ADJ
0 1960-01-01  29.37  10.362547
1 1960-02-01  29.41  10.348453
2 1960-03-01  29.41  10.348453
3 1960-04-01  29.54  10.302911
4 1960-05-01  29.57  10.292459


/tmp/ipykernel_248032/1852250754.py:30: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  cpi_base = np.float64(df_cpi.loc[df_cpi["Date"] == "2023-07-01", "CPI"])


In [3]:
# Combine FRED data and define real return for each month
df_fred = pd.merge(df_total_share_price_idx, df_dividends, left_on='Date', right_on='Date')
df_fred = pd.merge(df_fred, df_cpi, left_on='Date', right_on='Date')

df_fred['Real_Price_Index'] = df_fred['Total_Price_Index']*df_fred['CPI_ADJ']
df_fred['Real_Dividends_Index'] = df_fred['Total_Dividends_Index']*df_fred['CPI_ADJ']
df_fred['Real_Price_Index_Lag'] = df_fred['Real_Price_Index'].shift()

df_fred['Real_Month_Return'] = (df_fred['Real_Price_Index'] + df_fred['Real_Dividends_Index']) / df_fred['Real_Price_Index_Lag']
df_fred['Real_Total_Return'] = df_fred['Real_Price_Index']
df_fred['Real_Month_Return'].where(np.arange(len(df_fred))>0, 0).add(1).cumprod() * df_fred.loc[0, 'Real_Total_Return']
df_fred['log_Real_Total_Return'] = np.log(df_fred['Real_Total_Return'])
df_fred['log_Real_Month_Return'] = np.log(df_fred['Real_Month_Return'])
print(df_fred)

         Date  Total_Price_Index  Total_Dividends_Paid  Total_Dividends_Index  \
0  1960-01-01           3.418156                20.114               0.094052   
1  1961-01-01           3.295080                21.063               0.098489   
2  1962-01-01           3.675497                22.973               0.107420   
3  1963-01-01           3.502071                24.332               0.113775   
4  1964-01-01           4.050319                27.955               0.130716   
..        ...                ...                   ...                    ...   
57 2017-01-01         105.023548              1922.862               8.991182   
58 2018-01-01         124.243166              3011.279              14.080551   
59 2019-01-01         111.166161              2506.554              11.720489   
60 2020-01-01         130.905203              2397.311              11.209675   
61 2021-01-01         138.835736              2427.633              11.351459   

        CPI    CPI_ADJ  Rea

In [4]:
# Read in data files 
# GOP term flags
url = "https://github.com/llorracc/as.180.369/blob/main/materials/presidential-term-data/GOP_flag.csv?raw=true"
temp = pd.read_csv(url,dtype=str)
df_gop = pd.DataFrame(temp)

# Add a specific day (e.g., the first day of the month)
df_gop['Date'] = df_gop['Date'] + '.01'

# Convert the complete date string to a datetime object
#df_gop['Date'] = datetime.strptime(df_gop['Date'], '%Y-%m-%d')
df_gop['Date'] = pd.to_datetime(df_gop['Date'])
df_gop['Date'] = df_gop['Date'].dt.normalize()
df_gop['Date'] = df_gop['Date'].dt.floor('D')
print(df_gop.head)

<bound method NDFrame.head of            Date GOP
0    1871-01-01   1
1    1871-02-01   1
2    1871-03-01   1
3    1871-04-01   1
4    1871-05-01   1
...         ...  ..
1826 2023-03-01   0
1827 2023-04-01   0
1828 2023-05-01   0
1829 2023-06-01   0
1830 2023-07-01   0

[1831 rows x 2 columns]>


In [1]:
# Create new data frame that merges GOP flag with returns data
df = pd.merge(df_fred, df_gop, left_on='Date', right_on='Date')
print(df.head(5))

NameError: name 'pd' is not defined

In [1]:
# Plot real total returns. Shade GOP presidential terms
# convenience function
def plot_series(ax, df, index='Date', cols=['Real_Total_Return'], area='GOP'):
    # convert area variable to boolean
    df[area] = df[area].astype(int).astype(bool)
    # set up an index based on date
    df = df.set_index(keys=index, drop=False)
    # line plot
    df.plot(ax=ax, x=index, y=cols, color='blue', legend=False)
    # extract limits
    y1, y2 = ax.get_ylim()
    ax.fill_between(
        df[index].index, y1=y1, y2=y2, where=df[area], facecolor='tab:red', alpha=0.4
    )
    return ax

# set up figure, axis
f, ax = plt.subplots(figsize=(12, 6))
plot_series(ax, df)
ax.set_title('Returns are Lower Under GOP Presidencies', loc='left')

# ax.grid(visible=False)
ax.grid(axis='y')
ax.set_xlabel('')
ax.set_ylabel('Total Return\n(Real 2015 $)')

ax.set_yticks([40, 80, 120, 160])
# for text in ax.get_yticklabels():
#text.set_fontsize(40)

# plt.show()
# f.savefig('matts_plot.png')

NameError: name 'plt' is not defined

In [2]:
# Plot log total returns over time. Shade GOP presidential terms
# convenience function
def plot_series(ax, df, index='Date', cols=['log_Real_Month_Return'], area='GOP'):
    # convert area variable to boolean
    df[area] = df[area].astype(int).astype(bool)
    # set up an index based on date
    df = df.set_index(keys=index, drop=False)
    # line plot
    df.plot(ax=ax, x=index, y=cols, color='blue')
    # extract limits
    y1, y2 = ax.get_ylim()
    ax.fill_between(df[index].index, y1=y1, y2=y2, where=df[area], facecolor='grey', alpha=0.4)
    return ax

# set up figure, axis
f, ax = plt.subplots()
plot_series(ax, df)
ax.grid(True)
plt.show()

NameError: name 'plt' is not defined

In [3]:
# Run OLS regression of Monthly earnings onto GOP president flag
# Outcome real earnings next period. GOP president is pre-determined. 
df['GOPlag'] = df['GOP'].shift()

results = sm.formula.ols(formula = 'log_Real_Month_Return ~ GOPlag', data=df).fit(cov_type='HC3')
print(results.summary())

NameError: name 'df' is not defined